### Step 1: Import all required functions and libraries

In [1]:
# Import all required functions and libraries
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import (Conv2D, Dense, BatchNormalization, 
                                     BatchNormalization, Flatten,  Reshape,
                                     Conv2DTranspose)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### Step 2: Build the discriminator

In [2]:
def discriminator_model():
    
    # Instantiates the discriminator using
    # the Keras sequential API
    discriminator = Sequential(name="Discriminator")

    # Adds a convolutional layer to the discriminator
    # (from 256 × 256 × 3 into 128 × 128 × 32 tensor)
    discriminator.add(Conv2D(32, kernel_size=3, name="Conv_1",
                           strides=2, activation='leaky_relu',
                           input_shape=(256, 256, 3), padding="same"))

    # Adds a second convolutional layer to the discriminator
    # (from 128 × 128 × 32 into 64 × 64 × 64 tensor)
    discriminator.add(Conv2D(64, kernel_size=3, strides=2, name="Conv_2",
                           activation='leaky_relu', padding="same"))

    # Adds a batch normalization layer for stable training
    discriminator.add(BatchNormalization(name="BN_1"))

    # Adds a third convolutional layer to the discriminator
    # (from 64 × 64 × 64 into 32 × 32 × 128 tensor)
    discriminator.add(Conv2D(128, kernel_size=3, strides=2, name="Conv_3",
                           activation='leaky_relu', padding="same"))

    # Adds another batch normalization layer for stable training
    discriminator.add(BatchNormalization(name="BN_2"))

    # Adds a fourth convolutional layer to the discriminator
    # (from 32 × 32 × 128 into 16 × 16 × 256 tensor)
    discriminator.add(Conv2D(256, kernel_size=3, strides=2, name="Conv_4",
                           activation='leaky_relu', padding="same"))

    # Adds another batch normalization layer for stable training
    discriminator.add(BatchNormalization(name="BN_3"))

    # Adds a fifth convolutional layer to the discriminator
    # (from 16 × 16 × 256 into 8 × 8 × 512 tensor)
    discriminator.add(Conv2D(512, kernel_size=3, strides=2, name="Conv_5",
                           activation='leaky_relu', padding="same"))

    # Adds another batch normalization layer for stable training
    discriminator.add(BatchNormalization(name="BN_4"))

    # Flattens the network
    # (from 8 × 8 × 512 into (32768, ) 1D tensor)
    discriminator.add(Flatten())

    # Adds the output dense layer with sigmoid activation
    discriminator.add(Dense(1, activation='sigmoid'))

    # Prints the discriminator summary
    discriminator.summary()

    return discriminator

In [3]:
discriminator_model()

Model: "Discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_1 (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 Conv_2 (Conv2D)             (None, 64, 64, 64)        18496     
                                                                 
 BN_1 (BatchNormalization)   (None, 64, 64, 64)        256       
                                                                 
 Conv_3 (Conv2D)             (None, 32, 32, 128)       73856     
                                                                 
 BN_2 (BatchNormalization)   (None, 32, 32, 128)       512       
                                                                 
 Conv_4 (Conv2D)             (None, 16, 16, 256)       295168    
                                                                 
 BN_3 (BatchNormalization)   (None, 16, 16, 256)     

### Step 3: Build the generator

In [4]:
def generator_model():

    # Instantiates the generator using
    # the Keras sequential API
    generator = Sequential(name="Generator")

    # Adds a dense layer that has a number of neurons = 128 × 32 × 32
    # Input shape is 9 which is equal to the size of the noise vector
    generator.add(Dense(128 * 32 * 32, input_shape=(9, )))

    # Reshapes the image dimensions to 32 × 32 × 128
    generator.add(Reshape((32, 32, 128)))

    # Adds an upsampling layer to double the image size 
    # from 32 x 32 to 64 x 64
    generator.add(Conv2DTranspose(64, kernel_size=3, strides=2, name="ConvTr_1",
                                padding='same', activation="relu"))

    # Adds a batch normalization layer for stable training
    generator.add(BatchNormalization(name="BN_1"))

    # Adds a second upsampling layer to double the image size 
    # from 64 x 64 to 128 x 128
    generator.add(Conv2DTranspose(32, kernel_size=3, strides=2, name="ConvTr_2",
                                padding='same', activation="relu"))

    # Adds another batch normalization layer for stable training
    generator.add(BatchNormalization(name="BN_2"))

    # Adds a third upsampling layer with strides=1
    # This time, image size will not change
    # Tensor shape will change from 128 x 128 x 32 to 128 x 128 x 16
    generator.add(Conv2DTranspose(16, kernel_size=3, strides=1, name="ConvTr_3",
                                padding='same', activation="relu"))

    # Adds another batch normalization layer for stable training
    generator.add(BatchNormalization(name="BN_3"))

    # Adds a fourth upsampling layer to double the image size 
    # from 128 x 128 to 256 x 256 which is the original shape.

    # We don't need to add upsampling layers again becasue
    # the image size of 256 x 256  is equal to the original image size.

    # At the final layer, we do not apply batch normalization and,
    # instead of ReLU, we use the tanh activation.

    # The number of filters in the final layer is equal to the
    # number of color channels
    generator.add(Conv2DTranspose(3, kernel_size=3, strides=2, name="ConvTr_4",
                                padding='same', activation="tanh"))

    # Prints the generator summary
    generator.summary()

    return generator

In [5]:
generator_model()

Model: "Generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 131072)            1310720   
                                                                 
 reshape (Reshape)           (None, 32, 32, 128)       0         
                                                                 
 ConvTr_1 (Conv2DTranspose)  (None, 64, 64, 64)        73792     
                                                                 
 BN_1 (BatchNormalization)   (None, 64, 64, 64)        256       
                                                                 
 ConvTr_2 (Conv2DTranspose)  (None, 128, 128, 32)      18464     
                                                                 
 BN_2 (BatchNormalization)   (None, 128, 128, 32)      128       
                                                                 
 ConvTr_3 (Conv2DTranspose)  (None, 128, 128, 16)      46

In [6]:
discriminator = discriminator_model()
generator = generator_model()

Model: "Discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_1 (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 Conv_2 (Conv2D)             (None, 64, 64, 64)        18496     
                                                                 
 BN_1 (BatchNormalization)   (None, 64, 64, 64)        256       
                                                                 
 Conv_3 (Conv2D)             (None, 32, 32, 128)       73856     
                                                                 
 BN_2 (BatchNormalization)   (None, 32, 32, 128)       512       
                                                                 
 Conv_4 (Conv2D)             (None, 16, 16, 256)       295168    
                                                                 
 BN_3 (BatchNormalization)   (None, 16, 16, 256)     

### Step 4: Compile the discriminator

In [7]:
# Compiles the discriminator
discriminator.compile(loss='binary_crossentropy', 
                      optimizer=Adam(learning_rate=0.001), 
                      metrics=['accuracy'])

### Step 5: Build the GAN (combined) model

In [8]:
# Combined model = Complete GAN model
def GAN_model(generator, discriminator):

    # Instantiates the GAN model using
    # the Keras sequential API
    GAN = Sequential(name="GAN")

    # Freezes the weights of the discriminator to avoid updating 
    # the discriminator’s weights during generator training
    discriminator.trainable = False

    # Combines both generator and discriminator
    GAN.add(generator)
    GAN.add(discriminator)

    return GAN

In [9]:
GAN = GAN_model(generator, discriminator)

### Step 6: Compile the GAN (combined) model

In [10]:
# Compiles GAN (combined) model
GAN.compile(loss='binary_crossentropy', 
            optimizer=Adam(learning_rate=0.001))

### Step 7: Define the train function

In [13]:
def train(epochs, batch_size=128, save_interval=50):

    # Adversarial ground truths

    # Labels (1s) for real images:
    real = np.ones((batch_size, 1))
    # Labels (0s) for fake images:
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # Gets a random batch of real images
        index = np.random.randint(0, 3670 , batch_size)
        images = flowers_train[index]

        # Generates a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, 9))
        gen_images = generator.predict(noise)

        # Trains the discriminator 
        # Real images are classified as 1s and 
        # generated images are classified as 0s)
        d_loss_real = discriminator.train_on_batch(images, real)
        d_loss_fake = discriminator.train_on_batch(gen_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the combined model (generator)
        g_loss = GAN.train_on_batch(noise, real)

        # Outputs generated image samples at save_interval
        if epoch % save_interval == 0:
          plot_generated_images(generator)

In [14]:
def plot_generated_images(generator, image_grid_rows=3, image_grid_columns=3):

    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, 9))
    generated_images = generator.predict(z)
    generated_images = 0.5 * generated_images + 0.5
    fig, ax = plt.subplots(image_grid_rows, image_grid_columns,
                         figsize=(3, 3), sharey=True, sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            ax[i, j].imshow(generated_images[cnt, :, :, 0])
            ax[i, j].axis('off')
            cnt += 1

### Step 8: Load and preprocess the flowers dataset

In [15]:
# Load the flowers dataset using TFDS library
dataset, info = tfds.load('tf_flowers', split='train', with_info=True)

# Define a function to preprocess data
def preprocess(example):
    image = example['image']
    # Resize the image to 256 x 256
    image = tf.image.resize(image, [256, 256])
    # Convert the pixel values to [0, 1] to normalize data
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Apply preprocess function to each individual image and convert
# all images into a numpy array
images = []
for example in dataset:
    image = preprocess(example)
    images.append(image.numpy())
flowers_train = np.array(images)

# Print the shape of the array
print(flowers_train.shape) # (total_images, img_size, img_size, 3)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
(3670, 256, 256, 3)


### Step 9: Train the GAN model

In [ ]:
train(epochs=8000, batch_size=32, save_interval=50)

1/1 [==============================] - 0s 18ms/step
